In [1]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from matplotlib.ticker import ScalarFormatter
import numpy as np
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
from matplotlib.lines import Line2D

In [2]:
def plot_curve(x, y, xlab=None, ylab=None , title=None):
    '''
    plots a line graph
    
    imputs:
    x: varaible on x axis
    y: variable on y axis
    xlab: label on x axis
    ylab: label on y axis
    title: title of graph
    
    output:
    line graph
    '''
    plt.plot(x, y)
    if xlab is not None:
        plt.xlabel(xlab)
    if ylab is not None:
        plt.ylabel(ylab)
    if title is not None:
        plt.title(title)
    plt.show()

In [3]:
def plot_opt(t_opt, q, q_target, t):
    ''' 
    Plotting function to overlay optimal market execution time with target
    liquidation schedule
    
    imputs:
    t_opt: optimal time to execute depending on inventory size
    q: range of inventory
    q_target: the target inventory liquidation schedule
    t: the vector of times
    
    output:
    plot of overlay of optimal liquidation time and target schedule
    '''
    # Plot q vs t_opt
    line1, = plt.plot(t_opt, q, 'bo')
    
    # Plot q_target vs t
    line2, = plt.plot(t, q_target, 'k-', linewidth=2)
    plt.legend([line1, line2], ["Execute MO", "Target Liquidation Schedule"])
    plt.ylabel(r'Inventory ($q_t$)')
    plt.xlabel(r'Time ($sec$)')
    plt.show()

In [4]:
def plot_multi_lines(t, y, xlab=None, ylab=None, title=None):
    '''
    Function to produce a plot of how each row of y varies with t
    
    imputs:
    t: variable on x-axis
    y: matrix of values where each row will be plotted against t
    xlab: label for x-axis
    ylab: label for y-axis
    title: title of graph
    
    output:
    line graph of each row of y vs t
    '''
    
    # creating a vector so each row of y will be assigned a different colour
    color_idx = np.linspace(0, 1, y.shape[0])
    
    # actual plotting
    for i, line in zip(color_idx, range(0, y.shape[0], 1)):
        plt.plot(t, y[line, :], color=plt.cm.rainbow(i))
    if xlab is not None:
        plt.xlabel(xlab)
    if ylab is not None:
        plt.ylabel(ylab)
    if title is not None:
        plt.title(title)
    plt.show()

In [5]:
def plot_inventory(is_MO, Qpath, t):
    '''
    Gives a plot of the sample path of the liquidation of inventory with
    the points in time when market orders are executed
    
    imputs:
    is_MO: matrix indicating when market orders are executed for each sample path
    Qpath: matrix of sample paths of inventory trajectory
    t: vector of times
    
    output:
    plot of inventory trajectory over time with market orders overlaid
    '''
    
    # creating a vector so each row of Qpath will be assigned a different colour
    color_idx = np.linspace(0, 1, Qpath.shape[0])
    
    for i, line in zip(color_idx, range(0, Qpath.shape[0], 1)):
        plt.step(t, Qpath[line, :], color=plt.cm.rainbow(i))
        # step function used to plot due to Q being an integer
        
        specific_qpath = Qpath[line, :]
        thisisMO = is_MO[line, :].astype(bool)
        plt.plot(t[thisisMO], specific_qpath[thisisMO], 'bo')
        # plotting when market orders are executed for each trajectory
        
    plt.xlabel(r'Time ($sec$)')
    plt.ylabel(r'Inventory ($Q^*_t$)')
    plt.show()

In [6]:
def plot_multi_steps(x, y, xlab=None, ylab=None, title=None):
    '''
    Function to produce a plot of how each row of y varies with t
    
    imputs:
    x: variable on x-axis
    y: matrix of values where each row will be plotted against t
    xlab: label for x-axis
    ylab: label for y-axis
    title: title of graph
    
    output:
    step graph of each row of y vs t
    '''
        
    # creating a vector so each row of y will be assigned a different colour
    color_idx = np.linspace(0, 1, y.shape[0])
    
    
    for i, line in zip(color_idx, range(0, y.shape[0], 1)):
        plt.step(x, y[line, :], color=plt.cm.rainbow(i))
    if xlab is not None:
        plt.xlabel(xlab)
    if ylab is not None:
        plt.ylabel(ylab)
    if title is not None:    
        plt.title(title)
    plt.show()

In [7]:
def plot_price_per_share(t, pricePerShare, twap):
    '''
    Function to produce a plot of the average price per share vs TWAP
    
    imputs:
    t: vector of time elapsed
    pricePerShare: average price per share sold
    twap: Time Weighted Average Price
    
    output:
    line graph of the average price each share is executed for each
    sample path overlaid with TWAP
    '''
    
    # creating a vector so each row of pricePerShare will be assigned a different colour
    color_idx = np.linspace(0, 1, pricePerShare.shape[0])
    
    fig, ax = plt.subplots()
    for i, line in zip(color_idx, range(0, pricePerShare.shape[0], 1)):
        ax.plot(t, pricePerShare[line, :], color=plt.cm.rainbow(i), linestyle='-')
        ax.plot(t, twap[line, :], color=plt.cm.rainbow(i), linestyle='--')
        #plot of avg price per share for each sample path vs TWAP
        
    plt.ylabel(r'Price / Share ($\frac{X_t}{Q_t}$)')
    plt.xlabel(r'Time ($sec$)')
    plt.show()

In [8]:
class FormatScalarFormatter(matplotlib.ticker.ScalarFormatter):
    '''
    Class used in the fomratting of plot_heat_map
    '''
    def __init__(self, fformat="%1.1f", offset=True, mathText=True):
        self.fformat = fformat
        matplotlib.ticker.ScalarFormatter.__init__(self,useOffset=offset,
                                                        useMathText=mathText)
    def _set_format(self, vmin=None, vmax=None):
        self.format = self.fformat
        if self._useMathText:
            self.format = r'$\mathdefault{%s}$' % self.format

In [9]:
def plot_heat_map(t, q, myn_per_sim, meanq, medq, qAC, xlab=None, ylab=None, title=None):
    '''
    Plots a heat map of the trajectoy of the inventory from a 
    sample of inventory paths
    
    imputs:
    t: vector of time elapsed
    q: vector of inventory range
    myn_per_sim: 2D matrix of histograms
    meanq: mean inventory trajectory
    medq: medium inventory trajectory
    qAc: target schedule
    xlab: x label
    ylab: y label
    title: graph title
    
    output:
    heatmap
    '''
    
    # Sets where the ticks will be on the figure
    plt.tick_params(direction='in', bottom=True, top=True, left=True, right=True)

    x_cord, y_cord = np.meshgrid(t, q)
    # sets the corodinate grid
    
    cmap = plt.get_cmap('YlOrRd')
    # setting the colour of the heat map
    
    plot = plt.contourf(x_cord, y_cord, myn_per_sim, 100, cmap=cmap, levels=np.linspace(myn_per_sim.min(), myn_per_sim.max(), 1000))
    # produce contour plot to give frequency of occurance
    
    fmt = FormatScalarFormatter("%.2f")
    # set the format
    
    plt.colorbar(plot, format=fmt)
    # produce the colour bar

    plt.plot(t, meanq, linestyle='-', color='black', label=r'mean $Q_t^*$')
    plt.plot(t, medq, linestyle='--', color='black', label=r'median $Q_t^*$')
    plt.plot(t, qAC, linestyle='-', color='blue', label=r'target $q_t$')
    # add plots of mean, medium and target schedule
    
    if xlab is not None:
        plt.xlabel(xlab)
    if ylab is not None:
        plt.ylabel(ylab)
    if title is not None:
        plt.title(title)
    plt.legend()
    # adding axis and title labels
    plt.show()

In [10]:
def plot_histogram(x, xlab=None, prob=None, bins= None):    
    '''
    Produces a historgram
    
    imputs:
    x: numpy array
    xlab: x label
    prob: numpy array of percentiles
    bins: number of bins
    
    ouput:
    histogram
    '''
    # setting the bins of the histogram
    if bins is None:
        counts = plt.hist(x[~np.isnan(x)])
    else:
        counts = plt.hist(x[~np.isnan(x)], bins=bins)
    
    # adds a vertical line for a given percentile
    if prob is not None:    
        color_idx = np.linspace(0, 1, prob.shape[0])        
        q = np.quantile(x[~np.isnan(x)], prob)        
        for i, vline in zip(color_idx, range(0, prob.shape[0], 1)):          
            plt.axvline(x=q[vline], ymin=0, ymax=np.max(counts[0]), linestyle='--', color=plt.cm.rainbow(i), label='quantile ' + str(prob[vline]))
    
    # setting the x-label
    if xlab is not None:
        plt.xlabel(xlab)
    
    # setting y axis label
    plt.ylabel(r'Frequency')

    plt.legend()    
    plt.show()

In [11]:
def plot_3d(Z, x_axis, y_axis, xlab=None, ylab=None, zlab=None):
    '''
    Produces a 3D surface of a matrix
    
    imputs:
    Z: matrix looking to plot
    x_axis: values to be plotted on x axis
    y_axis: values to be plotted on y axis
    xlab: label on the x axis
    ylab: label on the y axis
    zlab: label on the z axis
    
    output:
    3D surface
    '''
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    
    # computing grid over which to plot surface
    X, Y = np.meshgrid(x_axis, y_axis)
    
    # adding axis labels 
    if xlab is not None:
        ax.set_xlabel(xlab)
    if ylab is not None:
        ax.set_ylabel(ylab)
    if zlab is not None:
        ax.set_zlabel(zlab)
        
    # plotting the surface
    ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,)

In [13]:
def plot_buy_orders(sim, t, sections=None, midpoint=None):
    '''
    Producing a scatter plot of the depth of buy
    orders recieved
    
    imputs:
    sim: 2D array of simulations of buy orders
    sections: adds horizontal lines to indicate different bins
    where depths can be grouped
    '''
    # for each simulation plotting the buy depths
    for k in range(sim.shape[0]):
        plt.scatter(t, sim[k,:], color='blue')

    # adds horizontal lines indicating the bins
    if sections is not None:
        for i in range(0,len(sections),1):
            plt.hlines(sections[i], xmin=t[0], xmax = t[-1], linestyles='--' )
            
    if midpoint is not None:
        for i in range(0,len(midpoint),1):
            plt.hlines(midpoint[i], xmin=t[0], xmax = t[-1], linestyles='--', color ='Red', label='Line 1')
            
    colors = ['Red']
    lines = [Line2D([0], [0], color=c, linewidth=2, linestyle='--') for c in colors]
    labels = ['Mid Point']
    plt.legend(lines, labels)
    
    plt.xlabel('time ($sec$)')
    plt.ylabel('buy order depth ($\$$)')
    
    # sets range of x values
    plt.xticks(t)
    
    # ensuring y axis starts at 0
    plt.gca().set_ylim(bottom=0)
    plt.show()